In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, Binarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

# Load data
train_labels_path = 'data/train_labels.csv'
train_values_path = 'data/train_values.csv'

train_labels = pd.read_csv(train_labels_path)
train_values = pd.read_csv(train_values_path)
data = train_values.merge(train_labels, on='building_id')

cols = ['age', 'height_percentage', 'area_percentage']

# Compute mean
mean_values = data[cols].mean()

# Compute Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data[cols].quantile(0.25)
Q3 = data[cols].quantile(0.75)

# Compute IQR
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = (data[cols] < lower_bound) | (data[cols] > upper_bound)
outlier_counts = outliers.sum()
outliers_mask = outliers.any(axis=1)

# Create two datasets: one with outliers, one without
data_with_outliers = data.copy()
data_without_outliers = data[~outliers_mask].copy()

# Print results
summary = pd.DataFrame({'Mean': mean_values, 'Outliers': outlier_counts})



# Drop unnecessary columns
data_with_outliers = data_with_outliers.drop(columns=['count_families', 'legal_ownership_status'])
data_without_outliers = data_without_outliers.drop(columns=['count_families', 'legal_ownership_status'])

def preprocess_and_train(data, label='With Outliers'):
    X = data.drop(columns=['damage_grade'])
    y = data['damage_grade']

    # Feature selection
    categorical_cols = [
        'land_surface_condition', 'foundation_type', 'roof_type',
        'ground_floor_type', 'other_floor_type', 'position',
        'plan_configuration'
    ]
    integer_cols = [
        'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
        'count_floors_pre_eq', 'age', 'area_percentage',
        'height_percentage'
    ]
    binary_cols = [
        'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
        'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
        'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
        'has_superstructure_timber', 'has_superstructure_bamboo',
        'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered'
    ]

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
    transformers=[
        ('int', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), integer_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols),
        ('bin', Binarizer(), binary_cols)
    ],
    remainder='passthrough', verbose_feature_names_out=False
)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Random Forest Pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=900, max_features=25, min_samples_split=17, random_state=42, n_jobs=-1, verbose=2))
    ])

    # Train model
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    micro_f1 = f1_score(y_test, y_pred, average='micro')

    print(f"{label} - Micro-F1 Score: {micro_f1:.4f}")
    return micro_f1

# Train on both datasets
score_with_outliers = preprocess_and_train(data_with_outliers, 'With Outliers')
score_without_outliers = preprocess_and_train(data_without_outliers, 'Without Outliers')

# Compare results
print("\n🔥 Comparison:")
print(f"With Outliers: {score_with_outliers:.4f}")
print(f"Without Outliers: {score_without_outliers:.4f}")



In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, Binarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

train_labels_path = 'data/train_labels.csv'
train_values_path = 'data/train_values.csv'

train_labels = pd.read_csv(train_labels_path)
train_values = pd.read_csv(train_values_path)
data = train_values.merge(train_labels, on='building_id')

categorical_cols = [
    'land_surface_condition', 'foundation_type', 'roof_type',
    'ground_floor_type', 'other_floor_type', 'position',
    'plan_configuration', 'legal_ownership_status'
]

integer_cols = [
    'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
    'count_floors_pre_eq', 'age', 'area_percentage',
    'height_percentage', 'count_families'
]

binary_cols = [
    'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
    'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
    'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
    'has_superstructure_timber', 'has_superstructure_bamboo',
    'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered'
]

data.drop(columns=['count_families', 'legal_ownership_status'])


X = data.set_index('building_id').drop(['damage_grade'], axis=1)
y = data.set_index('building_id')['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('int', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), integer_cols),
        ('cat', OrdinalEncoder(handle_unknown='error'), categorical_cols),
        ('bin', Binarizer(), binary_cols)
    ],
    remainder='passthrough', verbose_feature_names_out=False
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

class FuzzyRandomForest(RandomForestClassifier):
    def __init__(self, n_estimators=400, fuzzy_param=1, random_state=None, min_samples_split=2, verbose=2,
                 max_features=None, max_depth=None, min_samples_leaf=1, max_leaf_nodes=None, class_weight=None, n_jobs=None):
        super().__init__(
            n_estimators=n_estimators,
            random_state=random_state,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            verbose=verbose,
            class_weight=class_weight,
            n_jobs=n_jobs
        )
        self.fuzzy_param = fuzzy_param

    def predict_proba_fuzzy(self, X):
        proba = super().predict_proba(X)
        fuzzy_proba = proba ** self.fuzzy_param
        fuzzy_proba /= fuzzy_proba.sum(axis=1, keepdims=True)
        return fuzzy_proba

    def predict_fuzzy(self, X):
        fuzzy_proba = self.predict_proba_fuzzy(X)
        return np.argmax(fuzzy_proba, axis=1) + 1

# Micro-F1を評価指標として設定
micro_f1_scorer = make_scorer(f1_score, average='micro')

# Skip gridSearchCV because of exhaustive search (longer runtime)
#param_grid = {
#    'max_depth': [None, 10, 15],
#    'min_samples_leaf': [1, 3, 5],
#    'fuzzy_param': [1, 1.2, 1.5],
#    'class_weight': [None, 'balanced_subsample']
#}

#grid_search = GridSearchCV(
#    estimator=frf,
#    param_grid=param_grid,
#    scoring=micro_f1_scorer,
#    cv=3,
#    n_jobs=-1
#)
#best_model = grid_search.best_estimator_
#print("Best Params:", grid_search.best_params_)
#print("Best Micro-F1 (CV):", grid_search.best_score_)

frf = FuzzyRandomForest(
    n_estimators = 100,
    max_features = 25,
    min_samples_split = 17,
    random_state=42,
    n_jobs = -1
    )

frf.fit(X_train_processed, y_train)

y_pred_fuzzy = frf.predict_fuzzy(X_test_processed)
micro_f1_score_val = f1_score(y_test, y_pred_fuzzy, average='micro')

print("Test Micro-F1:", micro_f1_score_val)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.1s


building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100building tree 78 of 100

building tree 79 of 100
building tree 80 of 100
building tree 81

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   19.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s


Test Micro-F1: 0.740047197866503


[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


In [8]:
# Select the columns
cols = ['age', 'height_percentage', 'area_percentage']

# Compute mean
mean_values = data[cols].mean()

# Compute Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data[cols].quantile(0.25)
Q3 = data[cols].quantile(0.75)

# Compute IQR
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = (data[cols] < lower_bound) | (data[cols] > upper_bound)
outlier_counts = outliers.sum()

# Print results
summary = pd.DataFrame({'Mean': mean_values, 'Outliers': outlier_counts})
print(summary)


                        Mean  Outliers
age                26.535029     12499
height_percentage   5.434365      7843
area_percentage     8.018051     13557


In [9]:
outliers_mask = outliers.any(axis=1)

# Create two datasets: one with outliers, one without
data_with_outliers = data.copy()
data_without_outliers = data[~outliers_mask].copy()

print(f"Original data shape: {data.shape}")
print(f"Data without outliers shape: {data_without_outliers.shape}")


Original data shape: (260601, 40)
Data without outliers shape: (230632, 40)


In [20]:
X = data_without_outliers.set_index('building_id').drop(['damage_grade'], axis=1)
y = data_without_outliers.set_index('building_id')['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('int', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), integer_cols),
        ('cat', OrdinalEncoder(handle_unknown='error'), categorical_cols),
        ('bin', Binarizer(), binary_cols)
    ],
    remainder='passthrough', verbose_feature_names_out=False
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

class FuzzyRandomForest(RandomForestClassifier):
    def __init__(self, n_estimators=400, fuzzy_param=1, random_state=None, min_samples_split=2, verbose=2,
                 max_features=None, max_depth=None, min_samples_leaf=1, max_leaf_nodes=None, class_weight=None, n_jobs=None):
        super().__init__(
            n_estimators=n_estimators,
            random_state=random_state,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            verbose=verbose,
            class_weight=class_weight,
            n_jobs=n_jobs
        )
        self.fuzzy_param = fuzzy_param

    def predict_proba_fuzzy(self, X):
        proba = super().predict_proba(X)
        fuzzy_proba = proba ** self.fuzzy_param
        fuzzy_proba /= fuzzy_proba.sum(axis=1, keepdims=True)
        return fuzzy_proba

    def predict_fuzzy(self, X):
        fuzzy_proba = self.predict_proba_fuzzy(X)
        return np.argmax(fuzzy_proba, axis=1) + 1

# Micro-F1を評価指標として設定
micro_f1_scorer = make_scorer(f1_score, average='micro')


frf = FuzzyRandomForest(
    n_estimators = 500,
    max_features = 25,
    min_samples_split = 17,
    max_depth= 60,
    random_state=42,
    n_jobs = -1
    )

frf.fit(X_train_processed, y_train)

y_pred_fuzzy = frf.predict_fuzzy(X_test_processed)
micro_f1_score_val = f1_score(y_test, y_pred_fuzzy, average='micro')

print("Test Micro-F1:", micro_f1_score_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500building tree 23 of 500

building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500building tree 31 of 500

building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s


building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500
building tree 70 of 500
building tree 71 of 500
building tree 72 of 500
building tree 73 of 500
building tree 74 of 500
building tree 75 of 500
building tree 76 of 500
building tree 77 of 500
building tree 78 of 500
building tree 79 of 500
building tree 80

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   24.5s


...

[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   59.7s


building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500
building tree 395 of 500
building tree 396 of 500
building tree 397 of 500
building tree 398 of 500
building tree 399 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.8s


Test Micro-F1: 0.7440327790664903


[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.1s finished


In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, Binarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

# Split Data
X = data_without_outliers.set_index('building_id').drop(['damage_grade'], axis=1)
y = data_without_outliers.set_index('building_id')['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('int', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), integer_cols),
        ('cat', OrdinalEncoder(handle_unknown='error'), categorical_cols),
        ('bin', Binarizer(), binary_cols)
    ],
    remainder='passthrough', verbose_feature_names_out=False
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Define Fuzzy Random Forest Class
class FuzzyRandomForest(RandomForestClassifier):
    def __init__(self, n_estimators=400, fuzzy_param=1, random_state=None, min_samples_split=2, verbose=2,
                 max_features=None, max_depth=None, min_samples_leaf=1, max_leaf_nodes=None, class_weight=None, n_jobs=None):
        super().__init__(
            n_estimators=n_estimators,
            random_state=random_state,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            verbose=verbose,
            class_weight=class_weight,
            n_jobs=n_jobs
        )
        self.fuzzy_param = fuzzy_param

    def predict_proba_fuzzy(self, X):
        proba = super().predict_proba(X)
        fuzzy_proba = proba ** self.fuzzy_param
        fuzzy_proba /= fuzzy_proba.sum(axis=1, keepdims=True)
        return fuzzy_proba

    def predict_fuzzy(self, X):
        fuzzy_proba = self.predict_proba_fuzzy(X)
        return np.argmax(fuzzy_proba, axis=1) + 1

# Optuna Optimization
def objective(trial):
    # Define hyperparameter search space
    n_estimators = trial.suggest_int("n_estimators", 500, 700, step=25)
    max_features = trial.suggest_int("max_features", 5, X_train.shape[1])
    max_depth = trial.suggest_int("max_depth", 10, 100, step=10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 30)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Train Model
    model = FuzzyRandomForest(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(X_train_processed, y_train)
    y_pred_fuzzy = model.predict_fuzzy(X_test_processed)

    # Evaluate using Micro F1-score
    return f1_score(y_test, y_pred_fuzzy, average='micro')

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, n_jobs=-1)

# Print best parameters
print("Best Params:", study.best_params)
print("Best Micro-F1 Score:", study.best_value)

# Train final model with best parameters
best_params = study.best_params
best_model = FuzzyRandomForest(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train_processed, y_train)

# Final Evaluation
y_pred_fuzzy = best_model.predict_fuzzy(X_test_processed)
final_f1 = f1_score(y_test, y_pred_fuzzy, average='micro')

print("Final Test Micro-F1:", final_f1)


c:\Users\Bluewind\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-02-21 17:39:10,442] A new study created in memory with name: no-name-34a4c635-f96e-4df5-99aa-c3596424311f
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend T

building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 1 of 700
building tree 1 of 700
building tree 1 of 700
building tree 2 of 700
building tree 2 of 700
building tree 2 of 700
building tree 3 of 700
building tree 3 of 700
building tree 3 of 700
building tree 4 of 700
building tree 4 of 700
building tree 4 of 700
building tree 1 of 700
building tree 1 of 550
building tree 5 of 700
building tree 5 of 700
building tree 6 of 700
building tree 5 of 700
building tree 6 of 700
building tree 2 of 700
building tree 6 of 700
building tree 7 of 700
building tree 3 of 700
building tree 2 of 550
building tree 7 of 700
building tree 7 of 700
building tree 8 of 700
building tree 4 of 700
building tree 3 of 550
building tree 8 of 700
building tree 1 of 700
building tree 5 of 700
building tree 4 of 550
building tree 8 of 700
building tree 2 of 700
building tr

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   30.0s


building tree 34 of 650
building tree 35 of 650
building tree 33 of 700


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.5s


...

building tree 17 of 625
building tree 650 of 700
building tree 268 of 600
building tree 641 of 700
building tree 269 of 600
building tree 571 of 700
building tree 18 of 625
building tree 512 of 700
building tree 19 of 625
building tree 572 of 700
building tree 270 of 600
building tree 573 of 700
building tree 651 of 700
building tree 513 of 700
building tree 20 of 625
building tree 21 of 625
building tree 574 of 700
building tree 271 of 600
building tree 131 of 600
building tree 22 of 625
building tree 642 of 700
building tree 132 of 600
building tree 652 of 700
building tree 272 of 600
building tree 273 of 600
building tree 643 of 700
building tree 274 of 600
building tree 23 of 625
building tree 644 of 700
building tree 17 of 500
building tree 133 of 600
building tree 24 of 625
building tree 275 of 600
building tree 575 of 700building tree 276 of 600

building tree 134 of 600
building tree 18 of 500
building tree 19 of 500
building tree 653 of 700
building tree 20 of 500
building tre

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.7s


building tree 660 of 700
building tree 581 of 700
building tree 521 of 700
building tree 661 of 700
building tree 582 of 700
building tree 295 of 600
building tree 296 of 600
building tree 297 of 600
building tree 652 of 700
building tree 662 of 700
building tree 298 of 600
building tree 141 of 600
building tree 34 of 500
building tree 583 of 700
building tree 142 of 600
building tree 299 of 600
building tree 143 of 600
building tree 663 of 700
building tree 664 of 700
building tree 300 of 600
building tree 33 of 625
building tree 301 of 600


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   41.7s


building tree 35 of 500building tree 34 of 625
building tree 144 of 600
building tree 665 of 700

building tree 35 of 625
building tree 36 of 500
building tree 36 of 625
building tree 584 of 700
building tree 37 of 500
building tree 653 of 700
building tree 38 of 500
building tree 302 of 600
building tree 39 of 500
building tree 585 of 700
building tree 303 of 600
building tree 304 of 600
building tree 305 of 600
building tree 522 of 700
building tree 40 of 500
building tree 586 of 700
building tree 306 of 600
building tree 307 of 600
building tree 308 of 600
building tree 654 of 700
building tree 41 of 500
building tree 309 of 600
building tree 42 of 500
building tree 37 of 625
building tree 666 of 700
building tree 38 of 625
building tree 667 of 700
building tree 655 of 700
building tree 523 of 700
building tree 656 of 700
building tree 310 of 600building tree 39 of 625
building tree 43 of 500
building tree 657 of 700
building tree 44 of 500
building tree 311 of 600

building tree 31